# L-shape 算法的callback实现
采用上节的示例，利用回调函数的lazy constraint实现L-shape算法，此处将一阶段决策变量设定为整数变量。回调函数的书写参考JuMP官方文档示例。

In [2]:
using JuMP, Gurobi
import MathOptInterface as MOI
# parameters
# first stage
const c=[50,30,15,10]
const A=[-1 0 0 0;0 -1 0 0;0 0 -1 0;0 0 0 -1;0 -1 -1 -1]
const b=[-300,-700,-600,-500,-1600]
# second stage
const S=5
const p=[0.15,0.3,0.3,0.2,0.05]
const q=[-1150,-1525,-1900]
const W=[-6 -8 -10;-20 -25 -28;-12 -15 -18;-8 -10 -14;-1 0 0;0 -1 0;0 0 -1]
const T=[1 0 0 0;0 1 0 0;0 0 1 0;0 0 0 1;0 0 0 0;0 0 0 0;0 0 0 0]
const r1=[0,0,0,0,-15,-10,-5]
const r2=[0,0,0,0,-25,-20,-15]
const r3=[0,0,0,0,-25,-20,-25]
const r4=[0,0,0,0,-30,-25,-30]
const r5=[0,0,0,0,-10,-10,-10]
const r=[r1,r2,r3,r4,r5]

5-element Vector{Vector{Int64}}:
 [0, 0, 0, 0, -15, -10, -5]
 [0, 0, 0, 0, -25, -20, -15]
 [0, 0, 0, 0, -25, -20, -25]
 [0, 0, 0, 0, -30, -25, -30]
 [0, 0, 0, 0, -10, -10, -10]

In [ ]:
function solve_SP(x_value) #求解子问题，返回极射线或最优割
    β=zeros((4,))
    β0=0
    for s in 1:S
        SP=Model(Gurobi.Optimizer)
        set_silent(SP)
        set_attribute(SP,"InfUnbdInfo",1)
        set_attribute(SP,"DualReductions",0)
        @variable(SP,y[1:3]>=0)
        @objective(SP,Min, q'*y)
        @constraint(SP,cons, W*y.>=r[s]-T*x_value)
        optimize!(SP)
        if termination_status(SP)!=MOI.OPTIMAL
            unbd_ray=dual.(cons)
            β=vec(unbd_ray'*T)
            β0=unbd_ray'*r[s]
            return (is_opt=false,β=β,β0=β0)
        else
            duals=dual.(cons)
            β+=vec(p[s]*duals'*T)
            β0+=p[s]*duals'*r[s]
        end
    end
    return (is_opt=true,β=β,β0=β0)
end

solve_SP (generic function with 1 method)

In [ ]:
call_back_iter=0
function single_cut_callback(cb_data) #回调函数
    status=callback_node_status(cb_data,MP)
    if status!=MOI.CALLBACK_NODE_STATUS_INTEGER #仅在整数节点调用
        return
    end
    x_value=callback_value.(cb_data,x)
    result=solve_SP(x_value)
    global call_back_iter+=1
    println("Callback iteration: $call_back_iter")
    if result.is_opt
        cut=@build_constraint(result.β'*x+η>=result.β0) #最优割
        println("Adding optimality cut $cut")
    else
        cut=@build_constraint(result.β'*x>=result.β0) #可行割
        println("Adding feasibility cut $cut")
    end
    MOI.submit(MP,MOI.LazyConstraint(cb_data),cut)
end

single_cut_callback (generic function with 1 method)

In [ ]:
MP=Model(Gurobi.Optimizer)
set_silent(MP)
@variable(MP,x[1:4]>=0,Int)
@variable(MP,η)
@objective(MP,Min, c'*x+η)
@constraint(MP,A*x.>=b)
@constraint(MP,η>=-1e6) #初始松弛变量下界，否则问题无界
set_attribute(MP,MOI.LazyConstraintCallback(),single_cut_callback)#注册回调函数
optimize!(MP)

Set parameter LicenseID to value 2632898
Set parameter LicenseID to value 2632898
Set parameter LicenseID to value 2632898
Set parameter LicenseID to value 2632898
Set parameter LicenseID to value 2632898
Set parameter LicenseID to value 2632898
Adding optimality cut 
Callback iteration: 1
Adding optimality cut ScalarConstraint{AffExpr, MathOptInterface.GreaterThan{Float64}}(191.66666666666666 x[1] + η, MathOptInterface.GreaterThan{Float64}(-0.0))
Set parameter LicenseID to value 2632898
Set parameter LicenseID to value 2632898
Set parameter LicenseID to value 2632898
Set parameter LicenseID to value 2632898
Set parameter LicenseID to value 2632898
Adding optimality cut 
Callback iteration: 2
Adding optimality cut ScalarConstraint{AffExpr, MathOptInterface.GreaterThan{Float64}}(191.66666666666666 x[1] + η, MathOptInterface.GreaterThan{Float64}(-0.0))
Set parameter LicenseID to value 2632898
Set parameter LicenseID to value 2632898
Set parameter LicenseID to value 2632898
Set parameter 

In [14]:
println(value.(x))

[236.0, 688.0, 431.0, 318.0]
